In [1]:
# importing the required libraries
import pandas as pd
import re
from nltk.tokenize import word_tokenize
import nltk
from nltk.tag import pos_tag
from nltk import RegexpParser
from nltk import Tree
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')

In [2]:
def preprocess1(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent



In [3]:
def read_units():
    ml=[]
    with open("units_data/ml.txt") as f:
        ml =[wnl.lemmatize(re.sub("\n","",a).strip().lower()) for a in f.readlines()]
    
    gram = []
    with open("units_data/gram.txt") as f:
        gram = [wnl.lemmatize(re.sub("\n","",a).strip().lower()) for a in f.readlines()]

    return ml,gram

In [4]:
def divider(number):
    a_q = number.split("/")
    a1 = int(a_q[0])
    a2 = int(a_q[1])
    return a1/a2 


In [5]:
def convert_unit(substring4,unit,quantity_product,ml,gram):
    for m in ml:
        if m in substring4 and len(m)>0 and unit !="l":
            try:
                if unit == "kg":
                    return quantity_product,"l"
                else:
                    return quantity_product/1000,"l"
            except:
                
                return divider(quantity_product)/1000,"l"
    for g in gram:
        if g in substring4 and len(g)>0 and unit !="kg":
            try:
                if unit == "l":
                    return quantity_product,"kg"
                else:
                    return quantity_product/1000,"kg"
            except:
                return divider(quantity_product)/1000,"kg"
            
    return quantity_product,unit

In [6]:
def process(input_json_data,ml,gram):

    data = input_json_data
    df = None
    des,quantity,unit = [],[],[]
    bigram = ["seasoned dry"]
    unknow_words = ["assorted","free-range","hard","hard-cooked","added","no-salt",'chopped',"slices","pinch","large","medium","small","each","cooked","uncooked","cooled","pot", "pouch", "packed" ,"package", "regular", "roughly" ,"slice", "sliced", "strips"]
    units = [' ml ', ' cup ',' can ',' cans ',' litres ',' litre ', ' cups '," liter "," liters ", " cm "," in ",' g ', ' oz ',' lb ', ' kg ',' kgs ', ' cm ', ' tbsp ', " ounces "," lbs ",' tsp ', ' l ']
    units_remove = ['%','/','.','-','_',']','*','-/', 'c.', 'à', 'thé','hot','pkg','plus','tbsp/','litres','litre','half','and','cubed','cube', '/-inch', "in","cm",',packages', 'pitted', 'plain' ,'thin' ,'ea','-inch' ,'inch' ,'cube', 'cubes','thinly','tiny','finely','[','ml', 'cup',"cups",'can','cans', "liter","-in","liters", 'g', 'oz', 'kg', 'cm','lb','-lb' ,'tbsp', 'kgs','tsp', 'l',"ea","ounces","lbs","peeled","roasted"]

    for result in data['data']:
        d= result[u'ingredients'][u'ungrouped'][u'list']
        for a in d:
            actual_unit = a['unit']

            m= a['description']

            m = re.sub("g\.","g",m)
            m = re.sub("\. "," ",m)
            word_tokens = word_tokenize(m) 
            m  = (" ").join(word_tokens)
       
            split_string1 = m.split(',')

            done = 0
            if "," in m:
                patterns1 = """P: {<NN><,><NN>}
                                {<VBD><,><VBD>}
                                {<JJ><,><JJ>}    
                                {<VBN><,><VBN>}"""  
                PChunker1 = RegexpParser(patterns1)
                output_pattern1 = PChunker1.parse(preprocess1(m))
                for child in output_pattern1:
                    if isinstance(child, Tree):              
                        if child.label() == 'P':

                            if (child[0][1] == "VBD") and len(split_string1)<3:
                                done=1
                                substring2 = split_string1[0] + "," +split_string1[1]
                            elif child[0][1] == "NN" or child[0][1] == "VBN" or child[0][1] == "JJ":
                                done=1
                                substring2 = split_string1[0] + "," +split_string1[1]
            if done == 0:
                substring2 = split_string1[0]         
            substring2 = re.sub("and/or","or",substring2)
            substring2 = re.sub("or/and","or",substring2)
            split_string = substring2.split(' or ')
            substring1 = ""
            
            done = 0
            patterns = """P: {<JJ><CC><JJ>}
                             {<VBD><CC><VB>}
                             {<NN><CC><JJ>}
                             {<VBD><CC><JJ>}
                             {<CD><CC><CD>}""" 
            PChunker = RegexpParser(patterns)
            if " or " in substring2:
                found_pat = PChunker.parse(preprocess1(substring2))

                for child in found_pat:
                    if isinstance(child, Tree):               
                        if child.label() == 'P':
                            if child[1][0] == "or":
                                done=1
                                
                                substring1 = split_string[1]

            if done ==0:
                substring1 = split_string[0]

            substring1 = substring1.lower()
            index = 0
            u = "x"
            for x in units:
                finding = max(substring1.find(x),0)
                if finding > index:
                  index =finding
                  u = x
                  if actual_unit != None and u.strip() == actual_unit.strip():
                      break
                    
            if u == "x" and a['unit'] != None:
                u = a['unit']
            

            s = substring1
            pattern1 = r"(\d+){}".format(u)
            pattern2 = r"(\d+/\d+){}".format(u)
            pattern3 = r"(\d+.\d+){}".format(u)

            if len(re.findall(pattern2, s))>0:
                quantity_product = re.findall(pattern2, s)[0]

            elif len(re.findall(pattern1, s))>0:
                if len(re.findall(pattern3, s))>0:
                    quantity_product = re.findall(pattern3, s)[0]

                else:
                    quantity_product = re.findall(pattern1, s)[0]
            else:
                quantity_product = a['quantity']
                
            if index !=0:
                index = index + len(u.strip())+1
            



            substring3 = substring1
            substring3 = re.sub("\(.*?\)","",substring3)
            
            
            index = [i for i in range(len(substring3)) if substring3.startswith("(", i)]

            if len(index) == 0:
                pass
            else:
                index = index[0]
                substring3 = substring3[:index]  


            index = [i for i in range(len(substring3)) if substring3.startswith(")", i)]

            if len(index) == 0:
                pass
            else:
                index = index[-1]
                substring3 = substring3[index+1:]      
            x = re.findall("\d", substring3)

            for aaa in set(x):
                substring3 = re.sub(aaa,"",substring3)
            substring4 = substring3
            word_tokens = word_tokenize(substring4) 
            filtered_sentence = [wnl.lemmatize(w) for w in word_tokens if not w in unknow_words+units_remove] 
            substring4  = (" ").join(filtered_sentence)


            substring4 = substring4.strip(",")
            substring4 = substring4.strip()
             
            u = u.strip()
            

            try:
                quantity_product =float(quantity_product)
                if u == 'litres' or u == 'litre' or u == "liter" or u == "liters" or u == "ml" or u == "l":
                    if u == "ml":
                        quantity_product = quantity_product/1000
                    u="l"
                elif u == "kgs" or u == "kg" or u == "g":
                    if u == "g":
                        quantity_product = quantity_product/1000
                    u="kg"
            except:
                pass
                

            for bi in bigram:
                if bi in substring4:
                    substring4 = re.sub(bi,"",substring4)
            
            output_converter = convert_unit(substring4,u,quantity_product,ml,gram)

            quantity_product,u = output_converter[0],output_converter[1]

            des.append(substring4)
            quantity.append(quantity_product)
            unit.append(u)
            df =pd.DataFrame(zip(des,quantity,unit),columns=["food","quantity","unit"])

    return df

In [7]:
def add_spaces(text):
    return " "+text+" "

In [8]:
def create_category_list():
  bakery = []
  with open("categories_data/Bakery.txt") as f:
    bakery = [add_spaces(wnl.lemmatize(re.sub("\n","",a).strip().lower())) for a in f.readlines()]

  chilled = []
  with open("categories_data/Chilled.txt") as f:
    chilled = [add_spaces(wnl.lemmatize(re.sub("\n","",a).strip().lower())) for a in f.readlines()]

  beverages = []
  with open("categories_data/Beverages.txt") as f:
    beverages = [add_spaces(wnl.lemmatize(re.sub("\n","",a).strip().lower())) for a in f.readlines()]

  dairy = []
  with open("categories_data/Dairy_Eggs.txt") as f:
    dairy = [add_spaces(wnl.lemmatize(re.sub("\n","",a).strip().lower())) for a in f.readlines()]

  fruit = []
  with open("categories_data/Fruit_Vegetables.txt") as f:
    fruit = [add_spaces(wnl.lemmatize(re.sub("\n","",a).strip().lower())) for a in f.readlines()]

  grains = []
  with open("categories_data/Grains_Beans.txt") as f:
    grains = [add_spaces(wnl.lemmatize(re.sub("\n","",a).strip().lower())) for a in f.readlines()]

  herbs = []
  with open("categories_data/Herbs_Spices.txt") as f:
    herbs = [add_spaces(wnl.lemmatize(re.sub("\n","",a).strip().lower())) for a in f.readlines()]

  meat = []
  with open("categories_data/Meat_Fish _Alternatives.txt") as f:
    meat = [add_spaces(wnl.lemmatize(re.sub("\n","",a).strip().lower())) for a in f.readlines()]
    
  nuts = []
  with open("categories_data/Nuts_Seeds.txt") as f:
    nuts =[add_spaces(wnl.lemmatize(re.sub("\n","",a).strip().lower())) for a in f.readlines()]
    
  pantry = []
  with open("categories_data/Pantry.txt") as f:
    pantry = [add_spaces(wnl.lemmatize(re.sub("\n","",a).strip().lower())) for a in f.readlines()]

  return bakery,chilled,beverages, dairy, fruit,grains,herbs,meat,nuts,pantry

In [9]:
def name_cat(name,bakery,chilled,beverages, dairy, fruit,grains,herbs,meat,nuts,pantry):
  for a in bakery:
    if a in name and len(a)>0:
      return 0

  for a in chilled:
    if a in name and len(a)>0:
      return 1

  for a in beverages:
    if a in name and len(a)>0:
      return 2

  for a in dairy:
    if a in name and len(a)>0:
      return 3

  for a in fruit:
    if a in name and len(a)>0:
      return 4

  for a in grains:
    if a in name and len(a)>0:
      return 5

  for a in herbs:
    if a in name and len(a)>0:
      return 6

  for a in meat:
    if a in name and len(a)>0:
      return 7

  for a in nuts:
    if a in name and len(a)>0:
      return 8

  for a in pantry:
    if a in name and len(a)>0:
      return 9

  return 10

In [10]:
def main_function(input_json):
    
    
    ml,gram = read_units()
    df = process(input_json,ml,gram)
    df1 = df.groupby(["food","unit"])
    bakery,chilled,beverages, dairy, fruit,grains,herbs,meat,nuts,pantry = create_category_list()
    json_bakery = []
    json_chilled = []
    json_beverages=[] 
    json_dairy =[] 
    json_fruit=[]
    json_grains =[] 
    json_herbs = []
    json_meat =[]
    json_nuts =[]
    json_pantry =[]
    json_others = []
    for name,group in df1:
        new = {}
        new["ingredient"]=name[0]
        try:
            if name[1] == "l" and group["quantity"].sum()<1:
                new["unit"]="ml"
                new["quantity"]=float(group["quantity"].sum()*1000)
            elif name[1] =="kg" and group["quantity"].sum()<1:
                new["unit"]="g"
                new["quantity"]=float(group["quantity"].sum()*1000)
            else:
                new["unit"]=name[1]
                if round(group["quantity"].sum(),2) == int(group["quantity"].sum())+0.99:
                    new["quantity"] = round(group["quantity"].sum(),0)
                else:
                    new["quantity"]=round(group["quantity"].sum(),2)
        except:
            
            new["unit"]=name[1]
            total = 0
            
            for qw in group["quantity"]:
                a = qw.split("/")
                
                a1 = int(a[0])
                a2 = int(a[1])
                total = total + a1/a2
                
            if round(total,2) == 0.33:
                quantity = "1/3"
            else:
                out = (total).as_integer_ratio()
                quantity = str(out[0])+"/"+str(out[1])
                if out[0] == out[1]:
                    quantity = out[0]
            
            new["quantity"] = quantity
        cat = name_cat(add_spaces(name[0]),bakery,chilled,beverages, dairy, fruit,grains,herbs,meat,nuts,pantry) 
        if cat == 0:
            json_bakery.append(new)
        elif cat == 1:
            json_chilled.append(new)
        elif cat == 2:
            json_beverages.append(new)
        elif cat == 3:
            json_dairy.append(new)
        elif cat == 4:
            json_fruit.append(new)
        elif cat == 5:
            json_grains.append(new)
        elif cat == 6:
            json_herbs.append(new)
        elif cat == 7:
            json_meat.append(new)
        elif cat == 8:
            json_nuts.append(new)
        elif cat == 9:
            json_pantry.append(new)
        elif cat == 10:
            json_others.append(new)



    final = {"Pantry":json_pantry,"Beverages":json_beverages,"Fruits and Vegetables":json_fruit,
             "Meat,Fish and Alternatives":json_meat,"Dairy and Eggs":json_dairy,
             "Chilled":json_chilled,"Grains and Beans":json_grains,"Herbs and Spices":json_herbs,
             "Nuts and Seeds":json_nuts,"Bakery":json_bakery,"You may also need":json_others}
    return final

    

In [11]:

import json
f = open('total.json')
data = json.load(f)
aa = main_function(data)

In [12]:
with open("sample.json", "w",encoding="utf-8") as outfile: 
    json.dump(aa, outfile,indent=4,ensure_ascii=False)